In [ ]:
import sys
sys.path.append("../")

In [ ]:
import numpy as np
import pandas as pd
import qat.lang.AQASM as qlm
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#This cell loads the QLM solver.
#QLMaaS == False -> uses PyLinalg
#QLMaaS == True -> try to use LinAlg (for using QPU as CESGA QLM one)
from QQuantLib.utils.qlm_solver import get_qpu
QLMaaS = False
linalg_qpu = get_qpu(QLMaaS)

In [ ]:
from QQuantLib.utils.data_extracting  import get_results

## 1. Computation of VaR

For a given confidence level $\alpha$, $VaR_\alpha(X)$ is the smallest value $x$ such that 
$$P[X\leq x]\geq (1-\alpha).$$ To compute this quantity we can do a binary search.

### 1.1 Classical binary search


A binary search for $N = 2^n$ discrete probabilities works as follows:
- We begin by computing the cumulative sum for the first $2^{n-1}$ probabilities.
- If the probability is lower than the one we demand, we set $2^{n-1} = 2^{n-1}+2^{n-2}$ entries. Otherwise, we set $2^{n-1} = 2^{n-1}-2^{n-2}$ entries.
- Now we can start in the first step until $N = 1$, then we stop.
In the following cells we do this process in an iterative fashion to compute the VaR

We will do this process with the probability distribution from Black-Scholes equation.

For computing the cumulative sum we can compute the convolution of the probability density with a step function:

$$P[X\leq a] = \int_{-\infty}^{\infty} p(x)*f_{step}(a)dx.$$

Where 
$$f_{step}(a) = \begin{cases} 
      1 & x\leq a \\
      0 & x\gt a 
   \end{cases}
$$

We can use the **DensityProbability** class for doing this computation

In [ ]:
from probability_class import DensityProbability
#Configuration  of a probability density
probability_type = "Black-Scholes"

S0 = 2
r = 0.04
T = 300/365
sigma = 0.1
density_dict = {
    "s_0": S0,
    "risk_free_rate": r,
    "maturity": T,
    "volatility": sigma    
}


In [ ]:
bs_pdf = DensityProbability(probability_type, **density_dict)

In [ ]:
alpha = 0.05
index = 0
sign = 1
n_qbits = 5
x = np.linspace(1,3,2**n_qbits)
probability = bs_pdf.probability(x)

In [ ]:
alpha = 0.05
index = 0
sign = 1
lista = ['Probability']
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()
ax1.plot(x, probability, '-o')
#plt.plot(x, probability, '-o')

for i in range(1,n_qbits+1):
    index = index+sign*2**(n_qbits-i)
    step_function = np.array([1.0 for i in range(len(probability))])
    step_function[index:] = 0.0
    suma = np.sum(probability*step_function)
    ax2.plot(x, step_function, '-')
    #plt.axvline(x[index])
    if (suma<=1-alpha):
        sign = 1
    else:
        sign = -1
    lista.append('a = {}'.format(x[index]))
if (sign==1): 
    index = index+sign

print("El resultado es: ",index)
plt.legend(lista)
ax1.set_ylabel('P(x)')
ax2.set_ylabel('step functions', color='b')

In [ ]:
print(
    'The integral of the probability until index: ',
    index, 'is: ',np.sum(probability[:index]))
print('The VaR value is: ', x[index])
print('This value is lower than alpha = ', alpha, ': ', np.sum(probability[:index])>alpha)


### 1.2 Quantum formulation for VaR

For computing the VaR using **Amplitude Estimation** methods the binary search will be uses as before but the the integral of the probability of each step will be computed using the **AE** methods.

So we need to transform the computation in a suitable **AE** problem. For doing this we are going to use an operator for loading the probability density and we are going to create a quantum operator for loading a step function.

#### 1.2.1 step_array function

The **step_array** from **QQuantLib/DL/data_loading** package creates an Abstract Gate who loads into a quantum state a step function. The inputs of this function are:

* index: position where the step is produced.
* size : size of the array. It has to be a power of 2

The **step_array** function will create an array of *size* elements where first elements until the index will be ones and the rest will be zeros.

**BE AWARE:** For the index  position the element of the array **WILL BE ZERO** 

When the array  is created the **step_array** function will load it into the quantum state using the **load_array** from **QQuantLib/DL/data_loading** package .

Following cell show how to use the **step_array** function. 

In [ ]:
from QQuantLib.DL.data_loading import step_array, uniform_distribution

# Settings for the loading data 

# number of qbits
n_qbits = 3 
# Index for the step function remember the value for the index will be 0
index = 3 

#qlm routine
routine = qlm.QRoutine()
register = routine.new_wires(n_qbits+1)
#Create a uniform distribution
routine.apply(uniform_distribution(n_qbits),register[:n_qbits])
#Creates the operator for loading the step function
step_operator = step_array(index,2**n_qbits)
print("Abstract Gate for loading step function")
%qatdisplay step_operator --depth 2
#Apply the operator for loading the step function to the system
routine.apply(step_operator,register)
print("Complete routine")
%qatdisplay routine --depth 0

The first operator is the uniform distribution: basically apply a Hadamard gate over the first n-1 qbits, so the state will be:

$$|\Psi\rangle = \dfrac{1}{\sqrt{N}} \sum_{i=0}^{2^n-1} |i\rangle  \otimes |0\rangle$$

The second operator will load the step function so:


$$f_{|j\rangle}(|i\rangle) = \begin{cases} 
      1 & |i\rangle \lt |j\rangle \\
      0 & |i\rangle \geq |j\rangle  
   \end{cases}
$$

So the final state will be:

$$|\Psi\rangle = \dfrac{1}{\sqrt{N}} \sum_{i=0}^{2^n-1} |i\rangle  \otimes \left[f_{|j\rangle}\left(|i\rangle\right)|0\rangle + OtherTerms*|1\rangle\right]$$

We are not interested in the $|1\rangle$ terms only in the $|0\rangle$, so:

$$|\Psi\rangle = \dfrac{1}{\sqrt{N}} \sum_{i=0}^{2^n-1} |i\rangle  \otimes f_{|j\rangle}(|i\rangle)|0\rangle + OtherTerms$$

For a n=3 qbits with step in $|3\rangle$ the step function will be:


$$f_{|3\rangle}(|i\rangle) = \begin{cases} 
      1 & |i\rangle \lt |3\rangle \\
      0 & |i\rangle \geq |3\rangle  
   \end{cases}
$$

$$|\Psi\rangle = \dfrac{1}{\sqrt{2^3}}\left[|0\rangle +|1\rangle+|2\rangle\right] \otimes |0\rangle + OtherStaff \otimes |1\rangle$$

So, in this case, the probability for getting the state $|0\rangle$ on the last qbit will be the sum of the probabilities for getting $|0\rangle\otimes|0\rangle$ or $|1\rangle\otimes|0\rangle$ or $|2\rangle\otimes|0\rangle$: $\frac{1}{2^{3}}+\frac{1}{2^{3}}+\frac{1}{2^{3}}=\frac{3}{2^{3}}=0.375$

In [ ]:
#First we get the complete state simulation
results_loading, _, _, _ = get_results(routine, linalg_qpu=linalg_qpu)
results_loading

In [ ]:
#Now we get the probability of the last qbit
results_loading, _, _, _ = get_results(routine, linalg_qpu=linalg_qpu, qubits=[n_qbits])
print(results_loading)
print("Is the Probability of getting |0> in the last qbit 0.375? ",np.isclose(
    results_loading[results_loading.index==0]['Probability'].iloc[0], 
    0.375
))

So for create a suitable **AE** problem for VaR computations we need to load a density probability and the correspondient step function. Our library allows to do this loading of the density probability:
1. Using the **load_probability** function
2. Using the **load_array** function 

Both function are in **QQuantLib/DL/data_loading** package. 

#### 1.2.2 Density probability loading using load_probability function.

Following cells show how to use **load_probability** and **step_array** functions for creating a suitable **AE** problem for the VaR cumulative sum.

This loading data procedure can be used with all the **AE** algorithms except the **RQAE** one.

We are going to use the **Black-Schole** probability

In [ ]:
from QQuantLib.DL.data_loading import load_probability, step_array
#Configuration  of a probability density
probability_type = "Black-Scholes"

S0 = 2
r = 0.04
T = 300/365
sigma = 0.1
density_dict = {
    "s_0": S0,
    "risk_free_rate": r,
    "maturity": T,
    "volatility": sigma    
}
#Congfiguration of the porbability density
bs_pdf = DensityProbability(probability_type, **density_dict)
n_qbits = 5
x = np.linspace(1,3,2**n_qbits)
probability = bs_pdf.probability(x)
print("Probability density to load")
plt.plot(x, probability, '-o')

In [ ]:
# Now we create the QLM Routine for loading probability using load_probability
pl_routine = qlm.QRoutine()
# allocating qbits
pl_registers = pl_routine.new_wires(n_qbits+1)
# Creating the Abstract gate for loading probability density
p_gate = load_probability(probability)
print("Abstract Gate for Probability density loading")
%qatdisplay p_gate
# apply abstract gate for probability loading
pl_routine.apply(p_gate,pl_registers[:n_qbits])
# Our step function.
pl_index = 15
step_gate = step_array(pl_index, 2**n_qbits) 
print("Abstract Gate for step function loading")
%qatdisplay step_gate
pl_routine.apply(step_gate, pl_registers)

#For ploting the location of the step function in the probability density
print("Density Probability and Step function")
plt.plot(x, probability, 'o')
plt.axvline(x[pl_index])

In [ ]:
print("QLM circuit for sumulative sum copmputation")
%qatdisplay pl_routine

Now we can measure the state in order to test if the cumulative sum was properly loaded. In this case of daata loading we need to measure the last qbit and the probability of getting the $|0\rangle$ is the cumulative sum looked for.

In [ ]:
# For getting the results we need to measure the last qbit
results_loading, _, _, _ = get_results(pl_routine, linalg_qpu=linalg_qpu, qubits=[n_qbits])
results_loading

In [ ]:
measured_probability = results_loading[results_loading['Int_lsb'] ==0]['Probability'].iloc[0]
cum_sum = np.sum(probability[:pl_index])
print("Quantum Cumulative Sum: ", measured_probability)
print("Classical Cumulative Sum: ", cum_sum)
print('TEST: ', np.isclose(measured_probability, cum_sum))

#### 1.2.3 Density probability loading using load_array function.

Following cells show how to use **load_array** and **step_array** functions for creating a suitable **AE** problem for the VaR cumulative sum. 

This loading data procedure can be used with all the **AE** algorithms (including **RQAE**).

We are going to use the **Black-Scholes** probability

In [ ]:
from QQuantLib.DL.data_loading import load_array, uniform_distribution, step_array
from probability_class import DensityProbability
#Configuration  of a probability density
probability_type = "Black-Scholes"

S0 = 2
r = 0.04
T = 300/365
sigma = 0.1
density_dict = {
    "s_0": S0,
    "risk_free_rate": r,
    "maturity": T,
    "volatility": sigma    
}
#Congfiguration of the porbability density
bs_pdf = DensityProbability(probability_type, **density_dict)
n_qbits = 5
x = np.linspace(1,3,2**n_qbits)
probability = bs_pdf.probability(x)
print("Probability density to load")
plt.plot(x, probability, '-o')

In [ ]:
# Now we create the QLM Routine
al_routine = qlm.QRoutine()
# allocating qbits
al_registers = al_routine.new_wires(n_qbits+2)

# Loading Uniform Distribution over the first n qbits
al_routine.apply(uniform_distribution(n_qbits), al_registers[:n_qbits])

# Loding the probability as an array
al_p_gate = load_array(probability, id_name='prob')
al_routine.apply(al_p_gate, al_registers[:n_qbits+1])

print("Abstract Gate for Probability density loading")
%qatdisplay al_p_gate

#Step Function
al_index = 15

# Loading Step function
step_for_al = step_array(al_index, 2**n_qbits)
al_routine.apply(step_for_al, [al_registers[:n_qbits], al_registers[n_qbits+1]])
print("Abstract Gate for step function loading")
%qatdisplay step_gate

# Loading Uniform Distribution again over the first n qbits
al_routine.apply(uniform_distribution(n_qbits), al_registers[:n_qbits])

print("Density Probability and Step function")
plt.plot(x, probability, 'o')
plt.axvline(x[al_index])

In [ ]:
%qatdisplay al_routine

Now we can measure the state in order to test if the cumulative sum was properly loaded. In this data loading procedure the desired cumulative sum is loaded in the $|0\rangle^{n}\otimes |0\rangle \otimes |0\rangle$ stsate. The probability of this state is given by:

$$ a= \left|\dfrac{1}{N}\sum_{i=0}^{2^{n}-1}p(i)f(i)\right|^2,$$

where $\sum_{i=0}^{2^{n}-1}p(i)f(i)$ is the desired value we want to compute

In [ ]:
#Testing loading protocol
results_al, _, _, _ = get_results(al_routine, linalg_qpu=linalg_qpu)
cum_sum_al = np.sum(probability[:al_index])
measured_al_probability = np.sqrt(results_al['Probability'].iloc[0])*2**n_qbits
print("Quantum Cumulative Sum: ", measured_probability)
print("Classical Cumulative Sum: ", cum_sum)
print('TEST: ', np.isclose(measured_al_probability, cum_sum_al))

## 1.3 Amplitude Estimation Solution for VaR

Now we have adapted the cumulative sum VaR to an **AE**. Now we only need to use the different classes for getting results! 

### 1.3.1 Probabilty loading using load_probability function

For the data loading procedure using **load_probability** function we can use all the **AE** methods except the RQAE one.

In this case the target and the index for giving to the **AE** class wil be:

In [ ]:
target_for_pl = [0]
print('ae_target: ', target_for_pl)
index_for_pl = [pl_routine.arity-1]
print('ae_index: ', index_for_pl)

##### MLAE

In [ ]:
%%time
from QQuantLib.AE.maximum_likelihood_ae import MLAE

m_k = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
n_k = [100 for i in m_k]


mlae_dict = {
    'qpu': linalg_qpu,
    'schedule': [m_k, n_k],
    'mcz_qlm': True
}



mlae_co = MLAE(
    pl_routine,
    target = target_for_pl,
    index = index_for_pl, 
    **mlae_dict
)

mlae_co_a = mlae_co.run()

In [ ]:
cumsum_class = np.sum(probability[:pl_index])
print('mlae_a: ', mlae_co_a)
print('Cumulative Sum classical', cumsum_class)
print("Error is: ", np.abs(mlae_co_a-cumsum_class))

##### CQPEAE

In [ ]:
%%time
from QQuantLib.AE.ae_classical_qpe import CQPEAE

ae_cqpe_dict = {
    'qpu': linalg_qpu,
    'auxiliar_qbits_number': 10,
    'shots': 100,
    'mcz_qlm': True      
}


ae_cqpe = CQPEAE(
    pl_routine,
    target = target_for_pl,
    index = index_for_pl, 
    **ae_cqpe_dict
)
ae_cqpe_a  = ae_cqpe.run()

In [ ]:
cumsum_class = np.sum(probability[:pl_index])
print('ae_cqpe_a: ', ae_cqpe_a)
print('Cumulative Sum classical', cumsum_class)
print("Error is: ", np.abs(ae_cqpe_a-cumsum_class))

##### IQPEAE

In [ ]:
%%time
from QQuantLib.AE.ae_iterative_quantum_pe import IQPEAE

ae_iqpe_dict = {
    'qpu': linalg_qpu,
    'cbits_number': 8,
    'shots': 10,
    'mcz_qlm': True  
}

ae_iqpe = IQPEAE(
    pl_routine,
    target = target_for_pl,
    index = index_for_pl, 
    **ae_iqpe_dict
)

ae_iqpe_a  = ae_iqpe.run()

In [ ]:
cumsum_class = np.sum(probability[:pl_index])
print('ae_iqpe_a: ', ae_iqpe_a)
print('Cumulative Sum classical', cumsum_class)
print("Error is: ", np.abs(ae_iqpe_a-cumsum_class))

##### IQAE

In [ ]:
%%time
from QQuantLib.AE.iterative_quantum_ae import IQAE

iqae_dict = {
    'qpu': linalg_qpu,
    'mcz_qlm': True ,
    'epsilon': 0.001,
    'alpha': 0.05,
    'shots': 100,    
}

iqae = IQAE(
    pl_routine,
    target = target_for_pl,
    index = index_for_pl, 
    **iqae_dict
)

iqae_a = iqae.run()


In [ ]:
cumsum_class = np.sum(probability[:pl_index])
print('iqae_a: ', iqae_a)
print('Cumulative Sum classical', cumsum_class)
print("Error is: ", np.abs(iqae_a-cumsum_class))

### 1.3.1 Probabilty loading using load_array function

For the data loading procedure using **load_array** function we can use all the **AE** methods (including the RQAE one).

In this data loading procedure the desired cumulative sum is loaded in the $|0\rangle^{n}\otimes |0\rangle \otimes |0\rangle$ state. The probability of this state is given by:

$$ a= \left|\dfrac{1}{N}\sum_{i=0}^{2^{n}-1}p(i)f(i)\right|^2,$$

where $\sum_{i=0}^{2^{n}-1}p(i)f(i)$ is the desired value we want to compute

In this case the target and the index for giving to the **AE** class wil be:

In [ ]:
target_for_al = [0 for i in range(al_routine.arity)]
print('target_for_al: ', target_for_al)
index_for_al = [i for i in range(al_routine.arity)]
print('index_for_al: ', index_for_al)

##### MLAE

In [ ]:
%%time
from QQuantLib.AE.maximum_likelihood_ae import MLAE

m_k = [1, 10, 80, 90, 100, 110, 120]
n_k = [100 for i in m_k]


mlae_dict = {
    'qpu': linalg_qpu,
    'schedule': [m_k, n_k],
    'mcz_qlm': True
}



mlae_co = MLAE(
    al_routine,
    target = target_for_al,
    index = index_for_al, 
    **mlae_dict
)

mlae_co_a = mlae_co.run()

In [ ]:
cumsum_class = np.sum(probability[:al_index])
print('mlae_a: ', mlae_co_a)
cumsum_mlae = np.sqrt(mlae_co_a)*2**n_qbits
print('Cumulative Sum MLAE: ', cumsum_mlae)
print('Cumulative Sum classical', cumsum_class)
print("Error is: ", np.abs(cumsum_mlae-cumsum_class))

##### IQAE

In [ ]:
%%time
from QQuantLib.AE.iterative_quantum_ae import IQAE

iqae_dict = {
    'qpu': linalg_qpu,
    'mcz_qlm': True ,
    'epsilon': 0.0001,
    'alpha': 0.05,
    'shots': 100,    
}

iqae = IQAE(
    al_routine,
    target = target_for_al,
    index = index_for_al, 
    **iqae_dict
)

iqae_a = iqae.run()


In [ ]:
cumsum_class = np.sum(probability[:al_index])
cumsum_iqae = np.sqrt(iqae_a)*2**n_qbits
print('iqae_a: ', iqae_a)
print('Cumulative Sum IQAE: ', cumsum_iqae)
print('Cumulative Sum classical', cumsum_class)
print("Error is: ", np.abs(cumsum_iqae-cumsum_class))

##### RQAE

**BE AWARE!!** In the case of the RQAE method the desired value will be:

$$\sum_{i=0}^{2^{n}-1}p(i)f(i) = 2^n * a$$

Where a is the probabilty of the $|0\rangle^{n}\otimes |0\rangle \otimes |0\rangle$ state

In [ ]:
%%time
#Now we can apply de RQAE method
from QQuantLib.AE.real_quantum_ae import RQAE
q = 2
epsilon = 0.001
gamma = 0.05 
rqae_dict = {
    'qpu': linalg_qpu,
    'epsilon': epsilon,
    'gamma': gamma,
    'q': q,
    'mcz_qlm': True  
}



rqae = RQAE(
    al_routine,
    target = target_for_al,
    index = index_for_al, 
    **rqae_dict
)

_ = rqae.run()


In [ ]:
cumsum_class = np.sum(probability[:al_index])
cumsum_rqae = rqae.ae*2**n_qbits
print('rqae_a: ', rqae.ae)
print('Cumulative Sum RQAE: ', cumsum_rqae)
print('Cumulative Sum classical', cumsum_class)
print("Error is: ", np.abs(cumsum_rqae-cumsum_class))

##### CQPEAE

In [ ]:
%%time
from QQuantLib.AE.ae_classical_qpe import CQPEAE

ae_cqpe_dict = {
    'qpu': linalg_qpu,
    'auxiliar_qbits_number': 10,
    'shots': 10,
    'mcz_qlm': True      
}


ae_cqpe = CQPEAE(
    al_routine,
    target = target_for_al,
    index = index_for_al, 
    **ae_cqpe_dict
)
ae_cqpe_a  = ae_cqpe.run()

In [ ]:
cumsum_class = np.sum(probability[:pl_index])
cumsum_cqpeae = np.sqrt(ae_cqpe_a)*2**n_qbits
print('ae_cqpe_a: ', ae_cqpe_a)
print('Cumulative Sum CQPEAE: ', cumsum_cqpeae)
print('Cumulative Sum classical', cumsum_class)
print("Error is: ", np.abs(cumsum_cqpeae-cumsum_class))

##### IQPEAE

In [ ]:
%%time
from QQuantLib.AE.ae_iterative_quantum_pe import IQPEAE

ae_iqpe_dict = {
    'qpu': linalg_qpu,
    'cbits_number': 12,
    'shots': 10,
    'mcz_qlm': True  
}

ae_iqpe = IQPEAE(
    al_routine,
    target = target_for_al,
    index = index_for_al, 
    **ae_iqpe_dict
)

ae_iqpe_a  = ae_iqpe.run()

In [ ]:
cumsum_class = np.sum(probability[:pl_index])
cumsum_iqpeae = np.sqrt(ae_iqpe_a)*2**n_qbits
print('ae_iqpe_a: ', ae_iqpe_a)
print('Cumulative Sum IQPEAE: ', cumsum_iqpeae)
print('Cumulative Sum classical', cumsum_class)
print("Error is: ", np.abs(cumsum_iqpeae-cumsum_class))

Until now we have adapted the cumulative sum problem to an **AE** problem (for the 2 data loading procedures we have). For computing the **VaR** we only need to use the classical strategy and substitute the the way we perform the cumulative sum. 

We have developed a class for computing in an easy way this cumlative sum for the VaR (**CumulativeSumVaR** class) and a class for computing the VaR usig the beforementioned class.

In  notebook **04_Benchmark_Var** we show how this works